Test 8: Supply Voltage Sensitivity (PSRR Characterisation)

Objective: Characteerize the stability of the ADC readings as the supply voltage (VDD) varies across its allowed operating range (2.7V to 3.6V)

In [ ]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [ ]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

In [ ]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

In [ ]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x03) #Now, also RGBC enable / PON = 1, AEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

In [ ]:
#Function to test the sensor output
def read_clear():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high]):
        clear = (clear_high << 8) | clear_low
        return {'clear': clear}
    return None

In [ ]:
print("NOTE: Ensure the sensor is completely covered (DARK) for this test.")
time.sleep(2)

print("Starting voltage sweep...")
voltages = np.arange(2.7, 3.7, 0.1) #Datasheet specifies 2.7V to 3.6V
print(f"{'VDD(V)':<10} | {'CLEAR':<15}")
print("_" * 30)

for v in voltages:
    #1. Setting the voltages
    dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(v))
    dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))

    #2. Reading the sensor output
    samples = 5 #Taking 5 samples
    temp_sum = 0
    valid_reads = 0
    results = []
    for i in range(samples): 
        val = read_clear()
        if val is not None:
            temp_sum += val['clear']
            valid_reads += 1
            time.sleep(0.05)

        if valid_reads > 0: #taking average of valid reads
            avg_clear = temp_sum / valid_reads
            results.append(avg_clear)
            print(f"{v:<10} | {avg_clear:<15}")
        else:
            print(f"{v:<10} | {'Read Error':<15}")
            results.append(0)

    #3. Analysis
    if len(results) > 0:
        mean_avl = np.mean(results)
        std_dev = np.std(results)
        max_dev = np.max(results) - np.min(results)

        print("\n" + "+" * 40)
        print(f"Voltage Range:    2.7V - 3.6V")
        print(f"Mean Dark Count:  {mean_val:.2f}")
        print(f"Std Deviation:    {std_dev:.4f}")
        print(f"Max Deviation:    {max_dev:.2f}")

        if std_dev < 1.0:
            print("Result: PASS - Sensor output is stable under varying VDD.")
        else:
            print("Result: FAIL - Sensor output is unstable under varying VDD.")

print("Closing Device...")
time.sleep(0.5)
dwf.FDwfDeviceCloseAll()
print("Device Closed")